In [1]:
#If your are facing utf encoding problem in your dataset then use one of the below
#encoding = 'utf-8','ISO-8859-1','cp1252','latin1' or engine='python' use any one of these

#Import the Spam.csv dataset.

import pandas as pd
df = pd.read_csv("E:/datafiles/Spam.csv",encoding ='latin1')
df.head()

,Category,Message,Unnamed: 2
0,ham,"Go until jurong point, crazy.. Available only ...",NaN
1,ham,Ok lar... Joking wif u oni...,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN
3,ham,U dun say so early hor... U c already then say...,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN


In [2]:
#Now drop the Unnamed column from the dataframe 'df'.

df =df.drop('Unnamed: 2', 1)
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
#Now group it by category to know count/total of each categories are having
#Here we are having three categories 1:ham,2:ham/tYeah,3:spam

df.groupby('Category').describe()

Message         \
                                                     count unique   
Category                                                            
ham                                                   4825   4516   
ham\tHI BABE UAWAKE?FEELLIKW SHIT.JUSTFOUND OUT...       1      1   
ham\tYeah                                                1      1   
spam                                                   747    641   

                                                                                                       \
                                                                                                  top   
Category                                                                                                
ham                                                                            Sorry, I'll call later   
ham\tHI BABE UAWAKE?FEELLIKW SHIT.JUSTFOUND OUT...                                DEVIOUSBITCH.ANYWAY   
ham\tYeah                                                       give me a call if you've got a minute   
spam                                                Please call our customer service representativ...   

                                                         
                                                   freq  
Category                                                 
ham                                                  30  
ham\tHI BABE UAWAKE?FEELLIKW SHIT.JUSTFOUND OUT...    1  
ham\tYeah                                             1  
spam                                                  4

In [4]:
#The spam column is text and we know machine learning algorith doesnot understand text
# we need to convert both Category,Message columns from text to numbers/integers 
#By using pandas apply function and using lambda function on spam we can conert them into 0,1
#lambda function takes spam from dataset and verifies with if condition if x==spam then =1 else =0
#This value is loaded as spam into new spam column created in the dataset.

df['spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [5]:
#Now split_train_test your dataset by using sklearn.model_selection

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.Message,df.spam)

In [9]:
#Now even we have to convert Message column from text to numbers/integers
#This can be done by using CountVectorizer function

from sklearn.feature_extraction.text import CountVectorizer
corpus=[
    'This is the first documnet.',
    'This documnet is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names())
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

print(X.toarray())

['and', 'document', 'documnet', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
[[0 0 1 1 1 0 0 1 0 1]
 [0 1 1 0 1 0 1 1 0 1]
 [1 0 0 0 1 1 0 1 1 1]
 [0 1 0 1 1 0 0 1 0 1]]


In [11]:
from sklearn.feature_extraction.text import CountVectorizer.

v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count.toarray()[:3]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [12]:
#Remember that x_train_count is the message converted from text to numbers.

from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_count,y_train)

MultinomialNB()

In [13]:
#Now check your model with two sample emails wether your model predicting according to your model training

emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]
emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1], dtype=int64)

In [14]:
#Now test your model accuracy to do that convert your x_test into count bcz our model is designed to work on numbers
#After converting x_test to count then feed that x_test_count to model.score for prediction

X_test_count = v.transform(X_test)
model.score(X_test_count, y_test)

0.9885222381635581

In [15]:
#In order to avoid above method of converting x_test to count and using CountVectorizer then 
#convert the x-test count to a matrix by using v.fit.transform to an array of your dataset
#Then use the Sklearn Pipeline feature where you can define pipeline of your transformation
#In this process we are applying some transformation on our rawdata before feeding into our model

from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [16]:
#In previous model we converted x_test to count and after that we trained the model
#But here we can directly feed our test into model bcz internally the pipeline we created
#('vectorizer', CountVectorizer()) will convert our dataset to vector and then it will apply  ('nb', MultinomialNB())
#Navie_Bayes on that

clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [17]:
#Now to see your clf score.

clf.score(X_test,y_test)

0.9885222381635581

In [18]:
#Now to predict your clf.

clf.predict(emails)

array([0, 1], dtype=int64)